In [0]:
import matplotlib
matplotlib.use("Agg")

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import h5py

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
hdf5_path = '/content/gdrive/My Drive/GoogleCollab/Notebooks/DataAugumentation/snapshots.hdf5'
dataset = h5py.File(hdf5_path, "r")

In [0]:
train_labels=np.array(dataset["train_labels"])
train_labels = train_labels.reshape((len(train_labels),-1))
train_batch_labels=train_labels[train_batch_index]

In [0]:
train_batch_imgs=[]
for i in range(2500):
   img=(dataset['train_img'])[train_batch_index[i]]
   img=img/255.
   train_batch_imgs.append(img)    
train_batch_imgs=np.array(train_batch_imgs)

In [0]:
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [0]:
data = train_batch_imgs
labels = train_batch_labels

In [0]:
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels, 2)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [0]:
print("[INFO] performing 'on the fly' data augmentation")
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

[INFO] performing 'on the fly' data augmentation


In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

class ResNet:
	@staticmethod
	def residual_module(data, K, stride, chanDim, red=False,
		reg=0.0001, bnEps=2e-5, bnMom=0.9):
		# the shortcut branch of the ResNet module should be
		# initialize as the input (identity) data
		shortcut = data

		# the first block of the ResNet module are the 1x1 CONVs
		bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(data)
		act1 = Activation("relu")(bn1)
		conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False,
			kernel_regularizer=l2(reg))(act1)

		# the second block of the ResNet module are the 3x3 CONVs
		bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(conv1)
		act2 = Activation("relu")(bn2)
		conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride,
			padding="same", use_bias=False,
			kernel_regularizer=l2(reg))(act2)

		# the third block of the ResNet module is another set of 1x1
		# CONVs
		bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(conv2)
		act3 = Activation("relu")(bn3)
		conv3 = Conv2D(K, (1, 1), use_bias=False,
			kernel_regularizer=l2(reg))(act3)

		# if we are to reduce the spatial size, apply a CONV layer to
		# the shortcut
		if red:
			shortcut = Conv2D(K, (1, 1), strides=stride,
				use_bias=False, kernel_regularizer=l2(reg))(act1)

		# add together the shortcut and the final CONV
		x = add([conv3, shortcut])

		# return the addition as the output of the ResNet module
		return x

	@staticmethod
	def build(width, height, depth, classes, stages, filters,
		reg=0.0001, bnEps=2e-5, bnMom=0.9):
		# initialize the input shape to be "channels last" and the
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# set the input and apply BN
		inputs = Input(shape=inputShape)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(inputs)

		# apply CONV => BN => ACT => POOL to reduce spatial size
		x = Conv2D(filters[0], (5, 5), use_bias=False,
			padding="same", kernel_regularizer=l2(reg))(x)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = ZeroPadding2D((1, 1))(x)
		x = MaxPooling2D((3, 3), strides=(2, 2))(x)

		# loop over the number of stages
		for i in range(0, len(stages)):
			# initialize the stride, then apply a residual module
			# used to reduce the spatial size of the input volume
			stride = (1, 1) if i == 0 else (2, 2)
			x = ResNet.residual_module(x, filters[i + 1], stride,
				chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

			# loop over the number of layers in the stage
			for j in range(0, stages[i] - 1):
				# apply a ResNet module
				x = ResNet.residual_module(x, filters[i + 1],
					(1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

		# apply BN => ACT => POOL
		x = BatchNormalization(axis=chanDim, epsilon=bnEps,
			momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = AveragePooling2D((8, 8))(x)

		# softmax classifier
		x = Flatten()(x)
		x = Dense(classes, kernel_regularizer=l2(reg))(x)
		x = Activation("softmax")(x)

		# create the model
		model = Model(inputs, x, name="resnet")

		# return the constructed network architecture
		return model

In [0]:
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / EPOCHS)
model = ResNet.build(128, 128, 3, 2, (2, 3, 4),
	(32, 64, 128, 256), reg=0.0001)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [0]:
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit_generator(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network for 50 epochs...
Epoch 1/50
234/234 [==============================] - 279s 1s/step - loss: 0.2225 - acc: 0.9957 - val_loss: 0.2114 - val_acc: 1.0000
Epoch 2/50
234/234 [==============================] - 275s 1s/step - loss: 0.2053 - acc: 1.0000 - val_loss: 0.1999 - val_acc: 1.0000
Epoch 3/50
234/234 [==============================] - 274s 1s/step - loss: 0.1954 - acc: 1.0000 - val_loss: 0.1913 - val_acc: 1.0000
Epoch 4/50
234/234 [==============================] - 274s 1s/step - loss: 0.1878 - acc: 1.0000 - val_loss: 0.1846 - val_acc: 1.0000
Epoch 5/50
234/234 [==============================] - 272s 1s/step - loss: 0.1817 - acc: 1.0000 - val_loss: 0.1791 - val_acc: 1.0000
Epoch 6/50
234/234 [==============================] - 270s 1s/step - loss: 0.1767 - acc: 1.0000 - val_loss: 0.1744 - val_acc: 1.0000
Epoch 7/50
234/234 [==============================] - 265s 1s/step - loss: 0.1724 - acc: 1.0000 - val_loss: 0.1704 - val_acc: 1.0000
Epoch 8/50
234/234 [========

In [0]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.77      0.55      0.64       257
        dogs       0.64      0.83      0.72       243

    accuracy                           0.69       500
   macro avg       0.70      0.69      0.68       500
weighted avg       0.71      0.69      0.68       500



In [0]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('/content/gdrive/My Drive/GoogleCollab/Notebooks/DataAugumentation/dogs_vs_cats_small/plot.jpg')

In [0]:
print("[INFO] loading example image...")
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
image = load_img('/content/gdrive/My Drive/GoogleCollab/Notebooks/DataAugumentation/dogs_vs_cats_small/cats/cats_00001.jpg')
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

[INFO] loading example image...


In [0]:
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
total = 0

In [0]:
# construct the actual Python generator
print("[INFO] generating images...")
imageGen = aug.flow(image, batch_size=1, save_to_dir='/content/gdrive/My Drive/GoogleCollab/Notebooks/DataAugumentation/dogs_vs_cats_small/output/',
	save_prefix="image", save_format="jpg")

[INFO] generating images...
